In [1]:
# Importing libraries
from langchain_community.llms import Ollama

In [2]:
# Initialize the model
llm = Ollama(model = 'tinyllama')

C:\Users\himu23\AppData\Local\Temp\ipykernel_28472\90880870.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  llm = Ollama(model = 'tinyllama')


In [3]:
# Genrtae answers to a question 
question = "When did Gandhi Died?"
response = llm.invoke(question)

In [4]:
print(response)

Gandhi died on August 18, 1948 in Bombay (now Mumbai), India, at the age of 78 years old. He had been suffering from diabetes for several years and was advised by his doctors to take a short rest after he passed away.


In [5]:
# Genrtae answers to a question 
question = "Who was Sardar Vallabhbhai Patel?"
response = llm.invoke(question)
print(response)

Sarad Vallabhbhai Patel (1875-1950) was a prominent Indian politician who served as the first Indian home minister, and he played an essential role in the formation of India's independent government. He is best known for his leadership during India's struggle for independence from British colonial rule.

Patel, who was born in Mehsana, Gujarat, attended the University of Nagpur before completing his graduation from the London School of Economics and Political Science (LSE). He went on to serve as an Indian cabinet minister in several governments, including the first and second ministries led by Nehru. During the Quit India Movement in 1942, Patel was appointed a member of the All-India Congress Committee, and he played a crucial role in organizing its campaigns against British imperialism.

In addition to his political activities, Patel also served as an officer in the Indian Army during World War I. He was promoted to the rank of colonel in 1915 and rose through the ranks to become on

## Imptementing RAG for custom data

In [6]:
# !pip install langchain-community faiss-cpu sentence-transformers pypdf

In [31]:
!pip install "langchain==0.3.7" "langchain-community==0.3.7"

  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.7-py3-none-any.whl.metadata (2.9 kB)
  Using cached langchain_core-0.3.79-py3-none-any.whl.metadata (3.2 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached langsmith-0.1.147-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-1.26.4.tar.gz (15.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + D:\Anaconda\python.exe C:\Users\himu23\AppData\Local\Temp\pip-install-t5ctzxi1\numpy_40eccce4639f466bbcd9122b8166a135\vendored-meson\meson\meson.py setup C:\Users\himu23\AppData\Local\Temp\pip-install-t5ctzxi1\numpy_40eccce4639f466bbcd9122b8166a135 C:\Users\himu23\AppData\Local\Temp\pip-install-t5ctzxi1\numpy_40eccce4639f466bbcd9122b8166a135\.mesonpy-rqafd34r -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=C:\Users\himu23\AppData\Local\Temp\pip-install-t5ctzxi1\numpy_40eccce4639f466bbcd9122b8166a135\.mesonpy-rqafd34r\meson-python-native-file.ini
      The Meson build system
      Version: 1.2.99
      Source dir: C:\Users\himu23\AppData\Local\Temp\pip-install-t5ctzxi1\numpy_40eccce4639f466bbcd9122b8166a135
      Build dir: C:\Users\himu23\AppData\Local\Temp\pip-install-t5ctzxi1\numpy_40eccce4639f466bbcd9122b8

In [29]:
# --- Imports ---
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.prompts import ChatPromptTemplate

# --- Load and split PDF ---
pdf_reader = PyPDFLoader("physhology.pdf")
documents = pdf_reader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
chunks = text_splitter.split_documents(documents)

# --- Create embeddings and FAISS vector store ---
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = FAISS.from_documents(chunks, embedding=embeddings)

# --- Create retriever from FAISS ---
retriever = db.as_retriever()

# --- Initialize TinyLlama model via Ollama ---
llm = Ollama(model="tinyllama")

# --- Define condense question prompt ---
CONDENSE_QUESTION_PROMPT = ChatPromptTemplate.from_template("""
Given the following conversation and a follow-up question,
rephrase the follow-up question to be a standalone question.
Chat History: {chat_history}
Follow-up Input: {input}
Standalone question:
""")

# --- Create history-aware retriever ---
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    CONDENSE_QUESTION_PROMPT
)

# --- Create retrieval chain ---
qa = create_retrieval_chain(history_aware_retriever, llm)

# --- Example query ---
response = qa.invoke({
    "chat_history": [],
    "input": "Summarize the main psychological concepts from this PDF."
})

print(response["answer"])


ModuleNotFoundError: No module named 'langchain.chains'

In [8]:
pdf_reader = PyPDFLoader("physhology.pdf")
documents = pdf_reader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200,)
chunks = text_splitter.split_documents(documents)

In [9]:
# Create embeddings using free HF Model
embeddings = HuggingFaceEmbeddings()
db = FAISS.from_documents(documents=chunks, embedding = embeddings)

C:\Users\himu23\AppData\Local\Temp\ipykernel_28472\1410181046.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
C:\Users\himu23\AppData\Local\Temp\ipykernel_28472\1410181046.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()


In [10]:
# Initialize the model
llm = Ollama(model = 'tinyllama')

In [11]:
# Generate answers to a question 
question = "When did India get Nucler Missiles?"
response = llm.invoke(question)
print(response)

India got nuclear missiles in the early 1970s, after it signed the Nuclear Non-Proliferation Treaty (NPT) with other nations. The Indian nuclear program began in 1962 when it tested a bomb underground, but the government decided to wait until it became part of the NPT before pursuing nuclear weapons development.


In [14]:
import langchain
print(langchain.__version__)

1.0.3


In [17]:
# !pip install langchain langchain-community langchain-core

In [23]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.prompts import ChatPromptTemplate

CONDENSE_QUESTION_PROMPT = ChatPromptTemplate.from_template("""Given the following conversation and a follow question, rephrase the follow up question to be a standalone question.
    Chat History:{chat_history}
    Follow up Input:{question}
    Standalone question:""")
history_aware_retriever = create_history_aware_retriever(
    llm,
    retriever,
    CONDENSE_QUESTION_PROMPT
)

qa = create_retrieval_chain(history_aware_retriever, llm)

ModuleNotFoundError: No module named 'langchain.chains'